Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Descent Env"

In [17]:
import numpy as np
from descent_env import DescentEnv
import random 
from q_learning_agent import QLearningAgent

In [ ]:
# Cambiar render_mode a rgb_array para entrenar/testear
from env_recorder_wrapper import VideoRecorderWrapper 
import wandb
env = DescentEnv(render_mode='rgb_array')
recording_env = VideoRecorderWrapper(env, filename='recording.mp4', fps=3)

wandb.log({
    "gameplay_video_descent": wandb.Video("video/prueba.mp4", caption="Descent Gameplay", fps=3, format="mp4")
})
wandb.finish()

SyntaxError: invalid syntax (2096110540.py, line 2)

Observation Space

In [3]:
env.observation_space

Dict('altitude': Box(-inf, inf, (1,), float64), 'runway_distance': Box(-inf, inf, (1,), float64), 'target_altitude': Box(-inf, inf, (1,), float64), 'vz': Box(-inf, inf, (1,), float64))

Action Space

In [4]:
env.action_space

Box(-1.0, 1.0, (1,), float64)

Discretización de los estados

**Nota:** es importante que chequeen el espacio de observación y el espacio de acción del entorno. Los números usados son ejemplos y pueden no ser correctos

In [5]:
altitude_space = np.linspace(0, 2.5, 50)  
vertical_velocity_space = np.linspace(-3, 3, 100)
target_altitude_space = np.linspace(0, 1, 50) 
runway_distance_space = np.linspace(-1, 1, 50)


#print("altitude_space:", altitude_space)
#print("vertical_velocity_space:", vertical_velocity_space)
#print("target_altitude_space:", target_altitude_space)
#print("runway_distance_space:", runway_distance_space)


#import gymnasium as gym
#from descent_env import DescentEnv
#import numpy as np
#import random
#
#env = DescentEnv(render_mode="human") 
#obs, info = env.reset()
#
#num_episodes = 5 
#max_steps_per_episode = 500 
#
#all_altitudes = []
#all_vertical_velocities = []
#all_target_altitudes = []
#all_runway_distances = []
#
#for episode in range(num_episodes):
#    obs, info = env.reset()
#    done = False
#    steps = 0
#    print(f"\n--- Episodio {episode + 1} ---")
#    while not done and steps < max_steps_per_episode:
#        # Aquí generas una acción aleatoria válida para tu entorno
#        actions = [-1.0, -0.5, 0.0, 0.5, 1.0] # Acciones discretas sugeridas en el notebook
#        action = np.array([random.choice(actions)])
#
#        obs, reward, done, truncated, info = env.step(action)
#
#        # Recolectar datos
#        all_altitudes.append(obs["altitude"][0])
#        all_vertical_velocities.append(obs["vz"][0])
#        all_target_altitudes.append(obs["target_altitude"][0])
#        all_runway_distances.append(obs["runway_distance"][0])
#        steps += 1
#        
#    env.render()
#    env.close()
#
## Analizar los datos recolectados
#print("\n--- Análisis de rangos observados ---")
#print(f"Altitud: Min={np.min(all_altitudes):.2f}, Max={np.max(all_altitudes):.2f}")
#print(f"Velocidad Vertical: Min={np.min(all_vertical_velocities):.2f}, Max={np.max(all_vertical_velocities):.2f}")
#print(f"Altitud Objetivo: Min={np.min(all_target_altitudes):.2f}, Max={np.max(all_target_altitudes):.2f}")
#print(f"Distancia Pista: Min={np.min(all_runway_distances):.2f}, Max={np.max(all_runway_distances):.2f}")
#"""

In [6]:
actions = list(np.linspace(-1, 1, 10))
agent = QLearningAgent(
    altitude_space, 
    vertical_velocity_space, 
    target_altitude_space,
    runway_distance_space,  
    actions=actions,
    env=env
)

In [ ]:
rewards = agent.train_agent(env=env, episodes=25000, epsilon=0.99, gamma=0.99, alpha=0.1)

Episode: 0, Reward: -71.72, Epsilon: 0.99
Episode: 1, Reward: -82.08, Epsilon: 0.99
Episode: 2, Reward: -157.14, Epsilon: 0.99
Episode: 3, Reward: -103.42, Epsilon: 0.99
Episode: 4, Reward: -42.91, Epsilon: 0.99
Episode: 5, Reward: -160.48, Epsilon: 0.99
Episode: 6, Reward: -101.81, Epsilon: 0.99
Episode: 7, Reward: -118.02, Epsilon: 0.99
Episode: 8, Reward: -116.96, Epsilon: 0.99
Episode: 9, Reward: -74.50, Epsilon: 0.99
Episode: 10, Reward: -123.33, Epsilon: 0.99
Episode: 11, Reward: -118.20, Epsilon: 0.99
Episode: 12, Reward: -103.36, Epsilon: 0.99
Episode: 13, Reward: -81.21, Epsilon: 0.99
Episode: 14, Reward: -83.65, Epsilon: 0.99
Episode: 15, Reward: -126.08, Epsilon: 0.99
Episode: 16, Reward: -93.77, Epsilon: 0.99
Episode: 17, Reward: -89.17, Epsilon: 0.99
Episode: 18, Reward: -113.58, Epsilon: 0.99
Episode: 19, Reward: -94.41, Epsilon: 0.99
Episode: 20, Reward: -83.00, Epsilon: 0.99
Episode: 21, Reward: -131.72, Epsilon: 0.99
Episode: 22, Reward: -158.87, Epsilon: 0.99
Episode:

Obtener el estado a partir de la observación

In [ ]:
average_reward = np.mean(rewards)
print(f"Promedio de recompensas: {average_reward}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rewards)
plt.xlabel('Episodio')
plt.ylabel('Recompensa')
plt.title('Recompensas por episodio')
plt.show()

In [ ]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action